<a href="https://colab.research.google.com/github/stellacarv/Atividade-/blob/main/atvbigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Aluna:** Stella Carvalho

**Turma:** TADS37

# ***Exercícios PySpark***

In [4]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!pip install -q pyspark


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark-ETL").getOrCreate()
spark

In [6]:
import pyspark.sql.functions as F

data = spark.read.csv("sample_data/california_housing_train.csv", header=True)

print(f"The data contains: {data.count()} rows")
data.show()

The data contains: 17000 rows
+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|-114.310000|34.190000|         15.000000|5612.000000|   1283.000000|1015.000000| 472.000000|     1.493600|      66900.000000|
|-114.470000|34.400000|         19.000000|7650.000000|   1901.000000|1129.000000| 463.000000|     1.820000|      80100.000000|
|-114.560000|33.690000|         17.000000| 720.000000|    174.000000| 333.000000| 117.000000|     1.650900|      85700.000000|
|-114.570000|33.640000|         14.000000|1501.000000|    337.000000| 515.000000| 226.000000|     3.191700|      73400.000000|
|-114.570000|33.570000|         20.000000|1454.000000|    326.000000| 624.000000|

## **Atividades:**

# **1)**

In [7]:
data.orderBy(F.desc("housing_median_age")) \
    .select("housing_median_age", "median_house_value") \
    .show(10)


+------------------+------------------+
|housing_median_age|median_house_value|
+------------------+------------------+
|          9.000000|     158900.000000|
|          9.000000|      92600.000000|
|          9.000000|     209200.000000|
|          9.000000|     124700.000000|
|          9.000000|     158900.000000|
|          9.000000|      97800.000000|
|          9.000000|     157700.000000|
|          9.000000|     293100.000000|
|          9.000000|     119100.000000|
|          9.000000|     142600.000000|
+------------------+------------------+
only showing top 10 rows



Ao ordenar os dados pela idade mediana das casas, observa-se que as construções mais antigas (com housing_median_age igual a 52 anos) apresentam uma ampla variação de valores de mercado.
Algumas casas antigas possuem valores elevados, enquanto outras são relativamente baratas. Isso indica que a idade das construções não é o único fator determinante no preço dos imóveis.
O valor de mercado provavelmente está mais relacionado a fatores como localização, renda média da região, infraestrutura e demanda.

Em resumo, casas mais antigas não são necessariamente mais baratas, muitas vezes, imóveis antigos em áreas valorizadas mantêm ou até aumentam seu valor com o tempo.

# **2)**

In [10]:
from pyspark.sql import functions as F

# Cria categorias de renda baseadas na coluna median_income
data = data.withColumn(
    "income_category",
    F.when(F.col("median_income") < 2, "Baixa")
     .when((F.col("median_income") >= 2) & (F.col("median_income") < 4), "Média-baixa")
     .when((F.col("median_income") >= 4) & (F.col("median_income") < 6), "Média")
     .when((F.col("median_income") >= 6) & (F.col("median_income") < 8), "Média-alta")
     .otherwise("Alta")
)


In [11]:
data.groupBy("income_category").agg(
    F.avg("median_house_value").alias("avg_value"),
    F.count("*").alias("count")
).orderBy(F.desc("avg_value")).show()


+---------------+------------------+-----+
|income_category|         avg_value|count|
+---------------+------------------+-----+
|           Alta|460337.75423728814|  590|
|     Média-alta|345904.79354838707| 1395|
|          Média| 244047.7847940866| 4735|
|    Média-baixa| 168113.7251963746| 8275|
|          Baixa|111357.46184538654| 2005|
+---------------+------------------+-----+



As diferenças entre os valores médios das casas por categoria de renda refletem uma distribuição desigual, mostrando que áreas de maior renda possuem valores de imóveis mais altos.
O número de registros em cada categoria influencia a interpretação, pois categorias com poucos dados podem distorcer a média.
Embora a média ofereça uma visão geral, ela pode ser afetada por valores extremos; portanto, a mediana seria uma métrica mais adequada para representar o valor típico das casas nesse contexto.

# **3)**

In [12]:
data.select(
    F.corr("total_rooms", "median_house_value").alias("corr_rooms_value"),
    F.corr("latitude", "median_house_value").alias("corr_lat_value"),
    F.corr("longitude", "median_house_value").alias("corr_long_value")
).show()


+-------------------+-------------------+--------------------+
|   corr_rooms_value|     corr_lat_value|     corr_long_value|
+-------------------+-------------------+--------------------+
|0.13099146625326655|-0.1449167173376358|-0.04498169651090189|
+-------------------+-------------------+--------------------+



O número de quartos apresenta correlação positiva, indicando que casas com mais cômodos tendem a ter valores mais altos, embora a relação seja fraca.
A latitude e a longitude possuem correlação negativa: conforme a latitude aumenta (indo para o norte da Califórnia), o valor das casas tende a diminuir; e quanto maior a longitude (indo para o leste, afastando-se da costa), o preço também tende a cair.
Esses resultados refletem o contexto geográfico da Califórnia, onde regiões litorâneas e do sul, como Los Angeles e San Diego, possuem imóveis mais valorizados.